# Reproducing Vitaly Feldman MNIST Results

The goal of this notebook is to reproduce the results in https://arxiv.org/abs/2008.11193 which are based on the MNIST model from https://github.com/pytorch/opacus/blob/master/examples/mnist.py

In [48]:
import numpy as np

from syft.lib.adp.scalar import PhiScalar
from syft.lib.adp.entity import Entity
from syft.lib.adp.tensor import Tensor
from syft.lib.adp.adversarial_accountant import AdversarialAccountant
from syft.lib.adp.publish import publish

from torchvision import datasets, transforms
import torch
import torch as th

MNIST_MEAN = 0.1307
MNIST_STD = 0.3081
generator = None
sample_rate = 0.001

train_dataset = datasets.MNIST(
    './',
    train=True,
    download=True,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((MNIST_MEAN,), (MNIST_STD,)),
        ]
    ),
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    generator=generator,
#     **kwargs,
)

batch_size = 4

data_batch = torch.cat([train_dataset[i][0] for i in range(batch_size)])
label_batch = [train_dataset[i][1] for i in range(batch_size)]

In [49]:
class ScalarChainManagerTensor():
    """Supports convenience methods for scalar chains of abstraction"""
    
    def push_abstraction_top(self, scalar_type, *args, **kwargs):
        ""
        
class TensorChainManager():
    ""

In [50]:
class TensorChainM(PassthroughTensor):
    
    def __init__(self, child):
        self.child = child
        


In [51]:
from cocos import numerics as cn
import numpy as np
from passthrough import PassthroughTensor
from passthrough import implements

class SymEngineTensor(PassthroughTensor):
    """It turns out that when you're using symengine polynomials 
    there's a non-linear slowdown as you sum across larger and
    larger vectors. Subclassing from this class ensures that sums
    across large vectors are batched for faster performance"""
    
    
    def sum(self, dim, **kwargs):
        
        # when summing across symengine symbols vector sizes of 32 
        # seem to be optimal in terms of performance. there's a strange
        # non-linear increase in performance time when going higher
        batch_size = 512
        overall_shape = self.shape[dim]
        n_batches = int(overall_shape/batch_size)

        results = list()
        end = 0
        for i in range(n_batches-1):
            start = i*batch_size
            end = ((i+1)*batch_size)
            results.append(self.child[:,start:end,:].sum(dim))

        results.append(self.child[:,end:,:].sum(dim))
        out2 = sum(results)
        return SymEngineTensor(out2)
    
    def dot(self, other):
    
        expanded_self = self.child.repeat(other.shape[1]).reshape(self.shape[0], self.shape[1], other.shape[1])
        expanded_other = other.child.repeat(self.shape[0]).reshape(other.shape[0], other.shape[1], self.shape[0]).transpose(2,0,1)
        
        return SymEngineTensor(expanded_other * expanded_self).sum(1)

data = Tensor(data_batch.numpy())
target = Tensor(np.eye(10)[label_batch])

from syft.lib.adp.tensor import make_entities

entities = make_entities(n=len(data))
data = data.private(min_val=-0.42421296, max_val=2.8214867, entities=entities)
target = target.private(min_val=0, max_val=1, entities=entities)

data = data.reshape(-1,28*28)

weights = np.random.rand(28 * 28, 10)

# data = SymEngineTensor(data)
# weights = SymEngineTensor(weights)
# # d

making entities


In [52]:
%%timeit -n1 -r1
out = data.dot(weights)

1.13 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [54]:
out2 = np.dot(data,weights)

In [62]:
sp = out2[0][0].value.poly

In [ ]:
class ScalarTensor(np.ndarray):
    def __new__(
        cls,
        data
    ):
        obj = np.asarray(data).view(cls)
        return obj
    
    def __array_finalize__(self, obj):
        print("finalizing")
        if obj is None:
            return

    def __array_wrap__(self, out_arr, context=None):
        
        print(context)
        
        # out_arr.view calls __array_finalize__
        output = out_arr.view(self.__class__)
        
        return output

In [2]:
y = x_gpu.sum(0)

AttributeError: 'PlusIsMinusTensor' object has no attribute 'sum'

In [ ]:
y2 = x_cpu.sum(0)

In [6]:
y

PlusIsMinusTensor(child=[6])

In [7]:
y2

PlusIsMinusTensor(child=6)

In [20]:
x = DecoratorTensor([1,2,3,4])

finalizing


In [21]:
y = np.add(x,x)

finalizing
finalizing


In [12]:
x = PhiTensor([1,2,3])

In [15]:
y = x + x

In [16]:
y

Tensor([2, 4, 6])

In [4]:
import cocos.device as cd
from cocos.symbolic import (
    LambdifiedMatrixExpression, \
    find_length_of_state_vectors
)
cd.info()

Cocos running on ArrayFire v3.5.1 (OpenCL 64bit)
[0] Apple: AMD Radeon Pro Vega 56 Compute Engine | OpenCL | compute version 1.2


In [5]:
import sympy as sym
import numpy as np

In [8]:
# x1, x2, x3, t = sym.symbols('x1, x2, x3, t')
# argument_symbols = (x1, x2, x3)
# g = sym.Function('g')
# f = sym.Matrix([[x1 + x2], [(g(t) * x1 + x3) ** 2], [sym.exp(x1 + x2 + g(t))]])
# jacobian_f = f.jacobian([x1, x2, x3])

# def numeric_time_function(t: float):
#     return np.log(t)

# jacobian_f_lambdified \
#     = LambdifiedMatrixExpression(
#         argument_symbols=argument_symbols,
#         time_symbol=t,
#         symbolic_matrix_expression=jacobian_f,
#         symbolic_time_function_name_to_numeric_time_function_map={'g': numeric_time_function})

# n = 10000000
# X_gpu = cn.random.rand(n, 3)
# X_cpu = np.array(X_gpu)

In [9]:
# %%timeit
# result = X_gpu.sum()

In [10]:
# %%timeit
# result2 = X_cpu.sum()

In [37]:
single_poly = np.array([1,0,1])

single_poly.shape

(3,)

In [14]:
n_polys = (np.random.rand(10,3) > 0.5).astype(int)
n_polys

array([[1, 0, 0],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 0]])

In [12]:
x = data.flatten()[0:5]

In [1]:
import autograd.numpy as np   # Thinly-wrapped version of Numpy
from autograd import grad

def taylor_sine(x):  # Taylor approximation to sine function
    ans = currterm = x
    i = 0
    while np.abs(currterm) > 0.001:
        currterm = -currterm * x**2 / ((2 * i + 3) * (2 * i + 2))
        ans = ans + currterm
        i += 1
    return ans

grad_sine = grad(taylor_sine)
print("Gradient of sin(pi) is", grad_sine(x))

NameError: name 'x' is not defined

In [33]:
%%timeit
jacobian_f_numeric_gpu = \
    (jacobian_f_lambdified
     .evaluate_with_kwargs(x1=X_gpu[:, 0],
                           x2=X_gpu[:, 1],
                           x3=X_gpu[:, 2],
                           t=1.0))

NameError: name 'g' is not defined

In [24]:
%%timeit
jacobian_f_numeric_cpu = \
    (jacobian_f_lambdified
     .evaluate_with_kwargs(x1=X_cpu[:, 0],
                           x2=X_cpu[:, 1],
                           x3=X_cpu[:, 2],
                           t=1.0))

355 ms ± 5.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
print(f'numerical results from cpu and gpu match: '
      f'{np.allclose(jacobian_f_numeric_gpu, jacobian_f_numeric_cpu)}')

numerical results from cpu and gpu match: True


In [7]:
import symengine
from symengine import var

In [8]:
import uuid

In [9]:
syms = ""
for i in range(784):
    id = str(uuid.uuid4()).replace("-","")
    syms += 's'+id+str(i)+' '

In [10]:
v = var(syms)

In [11]:
%%timeit -n1 -r1
result = np.sum(v)

63.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
import pymbolic as pmbl

In [27]:
vs = list()
for i in range(784):
    x = pmbl.var("s"+str(i))
    vs.append(x)

In [28]:
%%timeit -n1 -r1
out = np.sum(vs)

2.12 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [68]:
syms = ""
for i in range(784):
    id = str(uuid.uuid4()).replace("-","")
    syms += 's'+id+str(i)+','

In [69]:
from sympy import symbols

In [70]:
symbols = symbols(syms[:-1])

In [71]:
%%timeit -n1 -r1
out = np.sum(symbols)

2.01 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [2]:
%%timeit -n1 -r1
out = np.sum([[x.value.poly for x in row] for row in data[0].tolist()])

2.05 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [3]:
# %%timeit -n1 -r1
# result = data[0].sum()

In [4]:
%%timeit -n1 -r1
result = data[1].sum()

2.06 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -n1 -r1
result = data[0].sum()

2.07 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%%timeit -n1 -r1
result = data[1].sum()

2.06 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
acc = AdversarialAccountant(max_budget=7000)

In [6]:
out = result.publish(acc=acc,sigma=1.5)
acc.print_ledger()

KeyboardInterrupt: 

In [48]:
acc.print_ledger()

<Entity:Constance_Weddle>	2.365443354618569


In [6]:
=

x = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=True)
y = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=False)

_weights = Tensor(np.random.uniform(size=(2,1)))

SyntaxError: invalid syntax (<ipython-input-6-07842543fea2>, line 1)

In [2]:
weights = _weights + 0
# acc = AdversarialAccountant(max_budget=7)  # causes error at end of budget


for i in range(3):
    batch_loss = 0

    pred = x.dot(weights)
    pre_loss = np.square(y-pred)
    loss = np.mean(pre_loss, axis=1)
#     loss = np.mean(pre_loss)    
    ledger = loss.backward(accumulate_grads=True)
    weight_grad = (weights.grad*0.3)
    weight_grad_noise = weight_grad.publish(acc=acc, sigma=0.005)
    weights = weights - weight_grad_noise
    ledger.zero_grads()
    batch_loss += loss.value
    print(np.sum([loss.value]))
    
acc.print_ledger()


0.17169736596498625
0.07441205306476974
0.04001946718047936
<Entity:George>	602.4247883171074
<Entity:Kritika>	602.4247883171074
<Entity:Madhava>	602.4247883171074
<Entity:Tudor>	602.4247883171074


In [13]:
x = PhiScalar(0,0.01,1)

In [14]:
out = x * x

In [15]:
out.publish(acc=acc)

<IntermediatePhiScalar: (0.0 < 0.000100000000000000 < 1.0)>
found one!


[0.703353940208477]

In [10]:
acc.print_ledger()

<Entity:George>	0.0
<Entity:Richard_Alvarez>	0.0002737137892581691


In [1]:
x.entity

NameError: name 'x' is not defined

In [7]:
import sympy as sym

In [9]:
a,b = sym.symbols('a,b')

In [10]:
y = a + b

In [38]:
from sympy.core.numbers import Number
class Float2(Number):
    
    def __init__(self, val):
        self.val = val
        
    def __add__(self, other):
        print("adding")
        return Float(self.val - other.val)

In [39]:
out = y.subs({a:Float2(3)})
out

b + 3

In [40]:
out.subs({b:Float2(2)})

5

In [9]:
# loss[0].value.poly

IndexError: too many indices for array

24ffce1fe8ab44a583abecf0b2e1ca69_c06e7e291fdb4cb6b5ec0b9abbabf1bf + 3*3175e5ebc4944304bb4a1a010fa5003d_162afbde9a87417a9a49c14568ea1c11 + 4*5ec1e3788c314340a6ea45342375281e_162afbde9a87417a9a49c14568ea1c11 + 6*68ed3172c1f6417faa4f5ca24611e12d_d59083b02ef74bdf8122e0c51454aa71 + 6*98d310c205b743be9af907ff4646238b_b63effdec6ba47bf93a126d0b5cc7086 + d082ac7660e84d0c806997c2dbe4f600_b63effdec6ba47bf93a126d0b5cc7086 + 5*eeffa282abe44326ae5fc1b8a6f8c6a0_d59083b02ef74bdf8122e0c51454aa71 + 2*fdefb3e070394d5cbf22820ac0ab7b00_c06e7e291fdb4cb6b5ec0b9abbabf1bf